## Load latest headlines

In [43]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-06-05-04-13-00 +0000,wsj,AUD’s Role as Economic Shock Absorber Might Fade,https://www.wsj.com/finance/currencies/asian-c...
1,2025-06-05-03-49-26 +0000,nyt,Drone Attacks Are the New Front in War. Can NA...,https://www.nytimes.com/2025/06/04/world/europ...
2,2025-06-05-03-22-03 +0000,nyt,Trump Travel Order Bans People From 12 Countri...,https://www.nytimes.com/2025/06/04/us/politics...
3,2025-06-05-03-22-00 +0000,wsj,Judge Blocks Deportation of Boulder Suspect’s ...,https://www.wsj.com/us-news/judge-blocks-depor...
4,2025-06-05-03-04-09 +0000,nyt,U.S. Brings Back Guatemalan Wrongly Deported t...,https://www.nytimes.com/2025/06/04/us/politics...


## Calculate word frequencies

In [44]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


C:\Users\CJ\AppData\Local\Temp\ipykernel_19180\2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,16
9,new,5
13,up,3
182,canada,3
58,putin,3


## Rank headlines by score

In [45]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-06-04-22-41-13 +0000,nyt,Trump and Putin Discuss Ukraine Drone Strike a...,https://www.nytimes.com/2025/06/04/us/politics...,34
50,2025-06-04-12-55-54 +0000,nyt,As Trump’s Tariffs Nudge Canada Toward Free In...,https://www.nytimes.com/2025/06/04/world/canad...,30
14,2025-06-04-22-19-00 +0000,wsj,Trump Administration Threatens to Pull Funding...,https://www.wsj.com/us-news/trump-administrati...,28
7,2025-06-05-02-00-00 +0000,wsj,Trump Halts New Foreign Student Enrollment at ...,https://www.wsj.com/us-news/education/trump-is...,27
21,2025-06-04-21-31-04 +0000,nyt,Putin Intends to Respond to Ukraine Strikes on...,https://www.nytimes.com/2025/06/04/world/europ...,27


## Select top headlines

In [46]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,34,2025-06-04-22-41-13 +0000,nyt,Trump and Putin Discuss Ukraine Drone Strike a...,https://www.nytimes.com/2025/06/04/us/politics...
32,20,2025-06-04-17-09-00 +0000,wsj,The Bank of Canada left its main interest rate...,https://www.wsj.com/articles/bank-of-canada-ke...
45,19,2025-06-04-13-48-50 +0000,nyt,"Jacinda Ardern, New Zealand’s Former Prime Min...",https://www.nytimes.com/2025/06/03/style/jacin...
26,15,2025-06-04-19-27-00 +0000,wsj,Some economists are beginning to question the ...,https://www.wsj.com/economy/cpi-inflation-data...
16,12,2025-06-04-22-07-03 +0000,nyt,Israeli-Backed Aid Sites in Gaza Close Tempora...,https://www.nytimes.com/2025/06/04/world/middl...
14,12,2025-06-04-22-19-00 +0000,wsj,Trump Administration Threatens to Pull Funding...,https://www.wsj.com/us-news/trump-administrati...
15,12,2025-06-04-22-08-32 +0000,nyt,Why South Korea’s New Leader Name Checked Nort...,https://www.nytimes.com/2025/06/04/world/asia/...
50,11,2025-06-04-12-55-54 +0000,nyt,As Trump’s Tariffs Nudge Canada Toward Free In...,https://www.nytimes.com/2025/06/04/world/canad...
48,10,2025-06-04-13-01-50 +0000,nyt,"Meet Rachel Hauck, the Set Designer Behind the...",https://www.nytimes.com/2025/06/04/arts/rachel...
0,9,2025-06-05-04-13-00 +0000,wsj,AUD’s Role as Economic Shock Absorber Might Fade,https://www.wsj.com/finance/currencies/asian-c...


## Next steps